In [12]:
import pandas as pd
from os.path import isfile

In [2]:
boroughs = ['Camden', 'City of London', 'Hackney', 'Hammersmith and Fulham', 'Haringey', 'Islington', 'Kensington and Chelsea', 'Lambeth', 'Lewisham', 'Newham', 'Southwark', 'Tower Hamlets', 'Wandsworth', 'Westminster', 'Barking and Dagenham', 'Barnet', 'Bexley', 'Brent', 'Bromley', 'Croydon', 'Ealing', 'Enfield', 'Greenwich', 'Harrow', 'Havering', 'Hillingdon', 'Hounslow', 'Kingston upon Thames', 'Merton', 'Redbridge', 'Richmond upon Thames', 'Sutton', 'Waltham Forest']

In [13]:
def convert_date(date):
    return '01/' + date[-2:] + '/' + date[:-2]

In [22]:
def process_crime_data(filepath):
    crime_data = pd.read_csv(filepath, usecols=lambda x: x != 'MinorText')

    # display(crime_data)

    output_file = "./converted-data/crime-data.csv"

    for date in crime_data.columns[2:]:
        data_subset = crime_data[['MajorText', 'LookUp_BoroughName', date]].copy()
        data_subset.rename(columns = {'MajorText': "Crime Type", "LookUp_BoroughName": 'Borough Name', date: 'Count'}, inplace=True)
        data_subset['Period'] = convert_date(date)
        grouped_data = data_subset.groupby(['Crime Type', 'Borough Name', 'Period'], as_index=False).sum()
        # grouped_data = pd.DataFrame(grouped_data)
        display(grouped_data)

        if not isfile(output_file):
            grouped_data.to_csv(output_file, index=False)
        else:
            grouped_data.to_csv(output_file, index=False, mode='a', header=False)


process_crime_data(r"C:\Users\felix\Futureproof-Laptop-Local\project-2\know-your-zone-api-sam\data\data-cleaning\raw-data\crime-data\MPS Borough Level Crime (most recent 24 months).csv")

,Crime Type,Borough Name,Count,Period
0,Arson and Criminal Damage,Barking and Dagenham,7,01/10/2020
1,Arson and Criminal Damage,Barking and Dagenham,120,01/10/2020
2,Arson and Criminal Damage,Barnet,5,01/10/2020
3,Arson and Criminal Damage,Barnet,153,01/10/2020
4,Arson and Criminal Damage,Bexley,6,01/10/2020
...,...,...,...,...
1119,Violence Against the Person,Wandsworth,179,01/10/2020
1120,Violence Against the Person,Wandsworth,395,01/10/2020
1121,Violence Against the Person,Westminster,0,01/10/2020
1122,Violence Against the Person,Westminster,247,01/10/2020


,Crime Type,Borough Name,Count,Period
0,Arson and Criminal Damage,Barking and Dagenham,4,01/11/2020
1,Arson and Criminal Damage,Barking and Dagenham,100,01/11/2020
2,Arson and Criminal Damage,Barnet,7,01/11/2020
3,Arson and Criminal Damage,Barnet,141,01/11/2020
4,Arson and Criminal Damage,Bexley,3,01/11/2020
...,...,...,...,...
1129,Violence Against the Person,Wandsworth,187,01/11/2020
1130,Violence Against the Person,Wandsworth,452,01/11/2020
1131,Violence Against the Person,Westminster,1,01/11/2020
1132,Violence Against the Person,Westminster,181,01/11/2020


,Crime Type,Borough Name,Count,Period
0,Arson and Criminal Damage,Barking and Dagenham,2,01/12/2020
1,Arson and Criminal Damage,Barking and Dagenham,110,01/12/2020
2,Arson and Criminal Damage,Barnet,6,01/12/2020
3,Arson and Criminal Damage,Barnet,144,01/12/2020
4,Arson and Criminal Damage,Bexley,7,01/12/2020
...,...,...,...,...
1115,Violence Against the Person,Wandsworth,164,01/12/2020
1116,Violence Against the Person,Wandsworth,405,01/12/2020
1117,Violence Against the Person,Westminster,1,01/12/2020
1118,Violence Against the Person,Westminster,199,01/12/2020


,Crime Type,Borough Name,Count,Period
0,Arson and Criminal Damage,Barking and Dagenham,4,01/01/2021
1,Arson and Criminal Damage,Barking and Dagenham,100,01/01/2021
2,Arson and Criminal Damage,Barnet,3,01/01/2021
3,Arson and Criminal Damage,Barnet,129,01/01/2021
4,Arson and Criminal Damage,Bexley,1,01/01/2021
...,...,...,...,...
1097,Violence Against the Person,Wandsworth,155,01/01/2021
1098,Violence Against the Person,Wandsworth,355,01/01/2021
1099,Violence Against the Person,Westminster,0,01/01/2021
1100,Violence Against the Person,Westminster,134,01/01/2021


,Crime Type,Borough Name,Count,Period
0,Arson and Criminal Damage,Barking and Dagenham,6,01/02/2021
1,Arson and Criminal Damage,Barking and Dagenham,104,01/02/2021
2,Arson and Criminal Damage,Barnet,7,01/02/2021
3,Arson and Criminal Damage,Barnet,122,01/02/2021
4,Arson and Criminal Damage,Bexley,0,01/02/2021
...,...,...,...,...
1108,Violence Against the Person,Wandsworth,150,01/02/2021
1109,Violence Against the Person,Wandsworth,382,01/02/2021
1110,Violence Against the Person,Westminster,1,01/02/2021
1111,Violence Against the Person,Westminster,129,01/02/2021


...

,Crime Type,Borough Name,Count,Period
0,Arson and Criminal Damage,Barking and Dagenham,5,01/05/2022
1,Arson and Criminal Damage,Barking and Dagenham,120,01/05/2022
2,Arson and Criminal Damage,Barnet,4,01/05/2022
3,Arson and Criminal Damage,Barnet,155,01/05/2022
4,Arson and Criminal Damage,Bexley,9,01/05/2022
...,...,...,...,...
1123,Violence Against the Person,Wandsworth,226,01/05/2022
1124,Violence Against the Person,Wandsworth,466,01/05/2022
1125,Violence Against the Person,Westminster,0,01/05/2022
1126,Violence Against the Person,Westminster,340,01/05/2022


,Crime Type,Borough Name,Count,Period
0,Arson and Criminal Damage,Barking and Dagenham,5,01/06/2022
1,Arson and Criminal Damage,Barking and Dagenham,141,01/06/2022
2,Arson and Criminal Damage,Barnet,14,01/06/2022
3,Arson and Criminal Damage,Barnet,154,01/06/2022
4,Arson and Criminal Damage,Bexley,2,01/06/2022
...,...,...,...,...
1121,Violence Against the Person,Wandsworth,218,01/06/2022
1122,Violence Against the Person,Wandsworth,412,01/06/2022
1123,Violence Against the Person,Westminster,1,01/06/2022
1124,Violence Against the Person,Westminster,333,01/06/2022


,Crime Type,Borough Name,Count,Period
0,Arson and Criminal Damage,Barking and Dagenham,9,01/07/2022
1,Arson and Criminal Damage,Barking and Dagenham,115,01/07/2022
2,Arson and Criminal Damage,Barnet,8,01/07/2022
3,Arson and Criminal Damage,Barnet,154,01/07/2022
4,Arson and Criminal Damage,Bexley,4,01/07/2022
...,...,...,...,...
1104,Violence Against the Person,Wandsworth,245,01/07/2022
1105,Violence Against the Person,Wandsworth,413,01/07/2022
1106,Violence Against the Person,Westminster,1,01/07/2022
1107,Violence Against the Person,Westminster,408,01/07/2022


,Crime Type,Borough Name,Count,Period
0,Arson and Criminal Damage,Barking and Dagenham,5,01/08/2022
1,Arson and Criminal Damage,Barking and Dagenham,133,01/08/2022
2,Arson and Criminal Damage,Barnet,5,01/08/2022
3,Arson and Criminal Damage,Barnet,138,01/08/2022
4,Arson and Criminal Damage,Bexley,15,01/08/2022
...,...,...,...,...
1115,Violence Against the Person,Wandsworth,176,01/08/2022
1116,Violence Against the Person,Wandsworth,364,01/08/2022
1117,Violence Against the Person,Westminster,1,01/08/2022
1118,Violence Against the Person,Westminster,391,01/08/2022


,Crime Type,Borough Name,Count,Period
0,Arson and Criminal Damage,Barking and Dagenham,4,01/09/2022
1,Arson and Criminal Damage,Barking and Dagenham,102,01/09/2022
2,Arson and Criminal Damage,Barnet,0,01/09/2022
3,Arson and Criminal Damage,Barnet,124,01/09/2022
4,Arson and Criminal Damage,Bexley,4,01/09/2022
...,...,...,...,...
1101,Violence Against the Person,Wandsworth,185,01/09/2022
1102,Violence Against the Person,Wandsworth,398,01/09/2022
1103,Violence Against the Person,Westminster,0,01/09/2022
1104,Violence Against the Person,Westminster,331,01/09/2022


In [ ]:
from os import listdir
from os.path import isfile, join

data_path = "./raw-data/crime-data"

datafile_names = listdir(data_path)
if len(datafile_names) != 1:
    raise ValueError("More than 1 crime data file found")
elif not isfile(join(data_path, datafile_names[0])):
    raise TypeError("No data file found")
else:
    datafile_path = join(data_path, datafile_names[0])
    process_crime_data(datafile_path)